In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
## imports and object instantiation
import json, time, nest_asyncio, json
from pystackql import StackQL
import pandas as pd
from IPython.display import clear_output, display, Markdown, HTML
from ipytree import Tree, Node

stackql = StackQL()
nest_asyncio.apply()

In [ ]:
## functions
def display_cards(cards_data):
    cards_html = ''
    
    for title, value in cards_data:
        card_template = f"""
        <div style="
            border: 1px solid #e3e3e3;
            border-radius: 4px;
            padding: 20px;
            display: inline-block;
            margin: 5px;
            text-align: center;
            width: 150px;
            background-color: #f7f7f7;">
            <h4 style="margin: 5px 0;">{title}</h4>
            <span style="font-size: 30px; font-weight: bold; color: red;">{value}</span>
        </div>
        """
        cards_html += card_template
    
    display(HTML(cards_html))

def get_icon(resType):
    if resType == "project":
        return 'codepen'
    else:
        return resType

def print_overwrite(message):
    clear_output(wait=True)
    print(message)

def build_tree_node(df, parent_name, parent_node=None):
    children = df[df['parentDisplayName'] == parent_name]
    
    for _, child in children.iterrows():
        child_node = Node(child['displayName'], opened=False, icon=get_icon(child['resType']))
        if parent_node:
            parent_node.add_node(child_node)
        build_tree_node(df, child['displayName'], child_node)

def explode_json_list_col(input_df, col_to_explode, exploded_col):
    output_df = input_df
    output_df[col_to_explode] = output_df[col_to_explode].apply(json.loads)
    output_df = output_df.explode(col_to_explode).rename(columns={col_to_explode: exploded_col})
    return output_df

In [ ]:
# get all folders and projects function
def get_resources_recursive(entity_id, get_projects_query_fn, get_folders_query_fn, parent_display_name='organization'):
    resources = []

    # Query for projects
    print_overwrite(f"Searching {entity_id} for projects...")
    project_query = get_projects_query_fn(entity_id)
    project_results = json.loads(stackql.execute(project_query))

    if isinstance(project_results, list):
        print_overwrite(f"Found {len(project_results)} projects in {entity_id}")
        for proj in project_results:
            proj["parentDisplayName"] = parent_display_name
            proj["resType"] = "project"
            resources.append(proj)

    # Query for folders
    print_overwrite(f"Searching {entity_id} for folders...")
    folder_query = get_folders_query_fn(entity_id)
    folder_results = json.loads(stackql.execute(folder_query))

    if isinstance(folder_results, list):
        print_overwrite(f"Found {len(folder_results)} folders in {entity_id}")
        for folder in folder_results:
            folder["parentDisplayName"] = parent_display_name
            folder["resType"] = "folder"
            resources.append(folder)

            # Fetch resources under this folder
            if 'name' in folder:
                resources.extend(get_resources_recursive(folder['name'], get_projects_query_fn, get_folders_query_fn, folder['displayName']))

    return resources